In [101]:
import pandas as pd
import sqlite3

# --- 1. Dati di esempio COMPLESSI ---

# Tabella 'positions'
# Ora include più titoli per lo stesso emittente (Apple) e gruppo (Volkswagen)
positions_data = {
    'fund_id': ['CGE_FUND', 'CGE_FUND', 'CGE_FUND', 'CGE_FUND', 'CGE_FUND'],
    'asset_ticker': ['AAPL', 'AAPL_BOND', 'VOW_DE', 'P911_DE', 'MSFT'],
    'quantity': [1000, 200, 800, 500, 1200],
    'market_value_eur': [150000, 210000, 900000, 450000, 250000]
}
df_positions = pd.DataFrame(positions_data)

# Tabella 'assets_master'
# Apple Inc. ha due ticker diversi. Volkswagen e Porsche sono nello stesso gruppo.
assets_data = {
    'ticker': ['AAPL', 'AAPL_BOND', 'X_TICKER', 'VOW_DE', 'P911_DE', 'MSFT'],
    'issuer_name': ['Apple Inc.', 'Apple Inc.', 'Azienda X', 'Volkswagen AG', 'Porsche AG', 'Microsoft Corp.'],
    'issuer_group': ['Apple Group', 'Apple Group', 'Gruppo X', 'Volkswagen Group', 'Volkswagen Group', 'Microsoft Group'],
    'is_ucits_eligible': [True, True, True, True, True, True]
}
df_assets = pd.DataFrame(assets_data)

# Tabella 'funds_master' (invariata)
funds_data = {
    'fund_id': ['CGE_FUND'],
    'nav_eur': [100000000] 
}
df_funds = pd.DataFrame(funds_data)


# --- 2. Creiamo il database e carichiamo le tabelle ---
conn = sqlite3.connect(':memory:')
df_positions.to_sql('positions', conn, index=False)
df_assets.to_sql('assets_master', conn, index=False)
df_funds.to_sql('funds_master', conn, index=False)

print("Database con dati complessi creato. Pronto per il test.")

# --- 3. Scrivi la tua query qui e testala ---

# Il ticker che il PM vuole acquistare
ticker_to_check = 'X_TICKER' 
# Il fondo su cui operare
fund_to_check = 'CGE_FUND'


# SOSTITUISCI QUESTA PARTE CON LA TUA QUERY
query = f"""

WITH
    issuer_x 
AS (Select 
        issuer_name, issuer_group 
    FROM 
        assets_master
    WHERE
        ticker = '{ticker_to_check}')
                
SELECT 
    SUM(CASE
            WHEN issuer_name = (SELECT issuer_name FROM issuer_x)
            THEN market_value_eur
            ELSE 0
            END),
    SUM(CASE
            WHEN issuer_group = (SELECT issuer_group FROM issuer_x)
            THEN market_value_eur
            ELSE 0
            END)
FROM
    assets_master AS a 
JOIN 
    positions AS p ON a.ticker = p.asset_ticker

WHERE
    p.fund_id = '{fund_to_check}';
"""


#
#
# Esegui la query e visualizza il risultato come DataFrame
result_df = pd.read_sql_query(query, conn)
print("\nRisultato della tua query:")
display(result_df)

# --- 4. Chiudi la connessione ---
conn.close()

Database con dati complessi creato. Pronto per il test.

Risultato della tua query:


,SUM(CASE\n WHEN issuer_name = (SELECT issuer_name FROM issuer_x)\n THEN market_value_eur\n ELSE 0\n END),SUM(CASE\n WHEN issuer_group = (SELECT issuer_group FROM issuer_x)\n THEN market_value_eur\n ELSE 0\n END)
0,0,0
